In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#import libraries
import pandas as pd
import csv
import os
from keras.preprocessing.text import text_to_word_sequence
import pandas as pd
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
import numpy as np
from keras.preprocessing import sequence
from io import StringIO
import nltk 
from nltk.corpus import stopwords
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer,PorterStemmer
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 
from sklearn.model_selection import train_test_split

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D,MaxPooling1D, Bidirectional,LSTM, SpatialDropout1D
!pip install keras-lookahead
from keras_lookahead import Lookahead

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

In [ ]:
#Load the dataset
review_data = pd.read_csv('../input/trip-advisor-hotel-reviews/tripadvisor_hotel_reviews.csv')
print(review_data.shape)
review_data.head()

In [ ]:
#remove the nulls
review_data['Review'].replace(' ', np.nan, inplace=True)
print(review_data['Review'].isnull().sum())
review_data['Rating'].replace(' ', np.nan, inplace=True)
print(review_data['Rating'].isnull().sum())

In [ ]:
review_data.info()

Text preprocessing

In [ ]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('x', '')
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) 
    return text

review_data['Review'] = review_data['Review'].apply(clean_text)

def getLemmText(text):
 tokens=word_tokenize(text)
 lemmatizer = WordNetLemmatizer()
 tokens=[lemmatizer.lemmatize(word) for word in tokens]
 return ' '.join(tokens)
review_data['Review'] = review_data['Review'].apply(getLemmText)


def getStemmText(text):
    tokens=word_tokenize(text)
    ps = PorterStemmer()
    tokens=[ps.stem(word) for word in tokens]
    return ' '.join(tokens)
review_data['Review'] = review_data['Review'].apply(getStemmText)

In [ ]:
review_data.Review[10]

In [ ]:
review_data['Rating'].unique()
#5 categories of rating

In [ ]:
#Tokenize the words, remove the symblos
#Convert the words to integers ising 'text to sequence'
tk = Tokenizer(num_words= 50000, filters = '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',lower=True, split=" ",)
tk.fit_on_texts(review_data['Review'])
x = tk.texts_to_sequences(review_data['Review'])

In [ ]:
#Encoding the integers using pad sequence 
#Adjust the number of words using maxlen based on word frequency 
X = sequence.pad_sequences(x, maxlen=200)

In [ ]:
#Y = review_data['Rating'].values
Y = pd.get_dummies(review_data.Rating)

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.10)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

In [ ]:
max_features = 50000
filters = 250
hidden_dims = 250
kernel_size = 2
dropout = 0.2

#Let us try using the Lookahead optimiser with Adam as the inner optimiser
opt = tf.keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=1e-6, amsgrad=False)
lk = Lookahead(opt, sync_period=5, slow_step=0.5, name = 'Lookahead')

model = Sequential()
model.add(Embedding(max_features,200,input_length=X_train.shape[1]))
model.add(SpatialDropout1D(0.3))
model.add(Conv1D(128,2, strides=1, padding='valid'))
model.add(MaxPooling1D(2,2))
model.add(Conv1D(256,2, strides=1, padding='valid'))
model.add(MaxPooling1D(2,2))
model.add(Bidirectional(LSTM(100, dropout=dropout, recurrent_dropout = dropout)))
model.add(Dropout(0.3))
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Dense(5))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=lk.optimizer,
              metrics=['accuracy'])
model.summary()
history = model.fit(X_train,Y_train,
          batch_size=32,
          epochs=10,validation_data = (X_test, Y_test)
          )

In [ ]:
# Accuracy
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Loss
plt.plot(history.history['loss'], label = 'loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.title('loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

#The model overfits, a possible reason could be the size of the dataset which is fairly small